In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
!pip install folium
import folium

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


 scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
gets = requests.get(url)
soup = BeautifulSoup(gets.content, 'html.parser')
table = soup.find('table')
t_r = table.find_all('tr')
rows = []
for tr in t_r:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':  #Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
        if neighborhood == 'Not assigned':  #If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough 
            neighborhood = borough   
        lst.append([postalcode, borough, neighborhood])

transform the data into a pandas dataframe

In [3]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)

groupby PostalCode and aggregating using .agg() to satisfy this condition"More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table."

In [4]:
df = df.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()

#### Reading CSV file from link "https://cocl.us/Geospatial_data" to get the latitude and longitude of each neighborhood and add them to our data frame

In [5]:
!wget -O GeoCord.csv http://cocl.us/Geospatial_data/

--2019-07-02 02:48:31--  http://cocl.us/Geospatial_data/
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/ [following]
--2019-07-02 02:48:31--  https://cocl.us/Geospatial_data/
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-07-02 02:48:31--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-07-02 02:48:31--  https://ibm.box.com/public/static/9afzr83pps4pwf2sm

In [6]:
df2= pd.read_csv('GeoCord.csv') # Reading csv

Assining postalcode accroding to coordinates -->using for loop 

In [7]:
df['Latitude'] = np.nan
df['Longitude'] = np.nan
for i in df.index:
    ci = df2['Postal Code'] == df.loc[i, 'PostalCode']
    df.at[i, 'Latitude'] = df2.loc[ci, 'Latitude'].values
    df.at[i, 'Longitude'] = df2.loc[ci, 'Longitude'].values

# generate maps to visualize your neighborhoods using folium

In [12]:
geolocator = Nominatim(user_agent="coursera")
add = 'Toronto'
try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(add, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(add, index))

map_ = folium.Map(location=[latitude, longitude], zoom_start=12)
#adding markers to map usin labels
for lat, lng, label in zip(df2['Latitude'], df2['Longitude'], df2['Postal Code']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3586cc',
        fill_opacity=0.5).add_to(map_)  
    
map_

The geograpical coordinates of Toronto are 43.653963, -79.387207.
